<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/separatemodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [2]:
import pandas as pd
import numpy as np

from ddop.newsvendor import KNeighborsWeightedNewsvendor
from ddop.metrics import average_costs

from sklearn.model_selection import GridSearchCV

from ddop.metrics import make_scorer
from ddop.metrics import prescriptiveness_score
from ddop.metrics import average_costs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Loading data-set

*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**



In [4]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

In [5]:
data

sell_price  year  is_snap_event  is_sporting_event  \
item_id date                                                             
0       2011-02-11         1.0     0              0                  0   
        2011-02-12         1.0     0              0                  0   
        2011-02-13         1.0     0              0                  0   
        2011-02-14         1.0     0              0                  0   
        2011-02-15         1.0     0              0                  0   
...                        ...   ...            ...                ...   
49      2016-05-18         2.5     5              0                  0   
        2016-05-19         2.5     5              0                  0   
        2016-05-20         2.5     5              0                  0   
        2016-05-21         2.5     5              0                  0   
        2016-05-22         2.5     5              0                  0   

                    is_national_event  is_cultural_event  is_religious_event  \
item_id date                                                                   
0       2011-02-11                  0                  0                   0   
        2011-02-12                  0                  0                   0   
        2011-02-13                  0                  0                   0   
        2011-02-14                  0                  1                   0   
        2011-02-15                  0                  0                   0   
...                               ...                ...                 ...   
49      2016-05-18                  0                  0                   0   
        2016-05-19                  0                  0                   0   
        2016-05-20                  0                  0                   0   
        2016-05-21                  0                  0                   0   
        2016-05-22                  0                  0                   0   

                    Friday  Monday  Saturday  ...  demand__minimum_twoweek  \
item_id date                                  ...                            
0       2011-02-11       1       0         0  ...                      0.0   
        2011-02-12       0       0         1  ...                      0.0   
        2011-02-13       0       0         0  ...                      0.0   
        2011-02-14       0       1         0  ...                      0.0   
        2011-02-15       0       0         0  ...                      0.0   
...                    ...     ...       ...  ...                      ...   
49      2016-05-18       0       0         0  ...                      6.0   
        2016-05-19       0       0         0  ...                      7.0   
        2016-05-20       1       0         0  ...                      7.0   
        2016-05-21       0       0         1  ...                      7.0   
        2016-05-22       0       0         0  ...                      7.0   

                    demand__sum_values_week  demand__median_week  \
item_id date                                                       
0       2011-02-11                     24.0                  3.0   
        2011-02-12                     26.0                  4.0   
        2011-02-13                     32.0                  4.0   
        2011-02-14                     30.0                  4.0   
        2011-02-15                     27.0                  4.0   
...                                     ...                  ...   
49      2016-05-18                    115.0                 18.0   
        2016-05-19                    111.0                 16.0   
        2016-05-20                    118.0                 18.0   
        2016-05-21                    119.0                 18.0   
        2016-05-22                    112.0                 17.0   

                    demand__mean_week  demand__standard_deviation_week  \
item_id date                              

## Split in feature and target matrix

In [6]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split

In [9]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

In [15]:
X_train

sell_price  year  is_snap_event  is_sporting_event  \
item_id date                                                             
0       2011-02-11        1.00     0              0                  0   
        2011-02-12        1.00     0              0                  0   
        2011-02-13        1.00     0              0                  0   
        2011-02-14        1.00     0              0                  0   
        2011-02-15        1.00     0              0                  0   
...                        ...   ...            ...                ...   
49      2014-12-27        2.68     3              0                  0   
        2014-12-28        2.68     3              0                  0   
        2014-12-29        2.68     3              0                  0   
        2014-12-30        2.68     3              0                  0   
        2014-12-31        2.68     3              0                  0   

                    is_national_event  is_cultural_event  is_religious_event  \
item_id date                                                                   
0       2011-02-11                  0                  0                   0   
        2011-02-12                  0                  0                   0   
        2011-02-13                  0                  0                   0   
        2011-02-14                  0                  1                   0   
        2011-02-15                  0                  0                   0   
...                               ...                ...                 ...   
49      2014-12-27                  0                  0                   0   
        2014-12-28                  0                  0                   0   
        2014-12-29                  0                  0                   0   
        2014-12-30                  0                  0                   0   
        2014-12-31                  0                  0                   0   

                    Friday  Monday  Saturday  ...  demand__minimum_twoweek  \
item_id date                                  ...                            
0       2011-02-11       1       0         0  ...                      0.0   
        2011-02-12       0       0         1  ...                      0.0   
        2011-02-13       0       0         0  ...                      0.0   
        2011-02-14       0       1         0  ...                      0.0   
        2011-02-15       0       0         0  ...                      0.0   
...                    ...     ...       ...  ...                      ...   
49      2014-12-27       0       0         1  ...                      0.0   
        2014-12-28       0       0         0  ...                      0.0   
        2014-12-29       0       1         0  ...                      0.0   
        2014-12-30       0       0         0  ...                      0.0   
        2014-12-31       0       0         0  ...                      0.0   

                    demand__sum_values_week  demand__median_week  \
item_id date                                                       
0       2011-02-11                     24.0                  3.0   
        2011-02-12                     26.0                  4.0   
        2011-02-13                     32.0                  4.0   
        2011-02-14                     30.0                  4.0   
        2011-02-15                     27.0                  4.0   
...                                     ...                  ...   
49      2014-12-27                     86.0                 15.0   
        2014-12-28                     89.0                 15.0   
        2014-12-29                     92.0                 15.0   
        2014-12-30                     88.0                 15.0   
        2014-12-31                     95.0                 15.0   

                    demand__mean_week  demand__standard_deviation_week  \
item_id date                              

In [16]:
X_test

sell_price  year  is_snap_event  is_sporting_event  \
item_id date                                                             
0       2015-01-01        0.98     4              1                  0   
        2015-01-02        0.98     4              1                  0   
        2015-01-03        0.98     4              1                  0   
        2015-01-04        0.98     4              1                  0   
        2015-01-05        0.98     4              1                  0   
...                        ...   ...            ...                ...   
49      2016-05-18        2.50     5              0                  0   
        2016-05-19        2.50     5              0                  0   
        2016-05-20        2.50     5              0                  0   
        2016-05-21        2.50     5              0                  0   
        2016-05-22        2.50     5              0                  0   

                    is_national_event  is_cultural_event  is_religious_event  \
item_id date                                                                   
0       2015-01-01                  1                  0                   0   
        2015-01-02                  0                  0                   0   
        2015-01-03                  0                  0                   0   
        2015-01-04                  0                  0                   0   
        2015-01-05                  0                  0                   0   
...                               ...                ...                 ...   
49      2016-05-18                  0                  0                   0   
        2016-05-19                  0                  0                   0   
        2016-05-20                  0                  0                   0   
        2016-05-21                  0                  0                   0   
        2016-05-22                  0                  0                   0   

                    Friday  Monday  Saturday  ...  demand__minimum_twoweek  \
item_id date                                  ...                            
0       2015-01-01       0       0         0  ...                      0.0   
        2015-01-02       1       0         0  ...                      0.0   
        2015-01-03       0       0         1  ...                      0.0   
        2015-01-04       0       0         0  ...                      0.0   
        2015-01-05       0       1         0  ...                      0.0   
...                    ...     ...       ...  ...                      ...   
49      2016-05-18       0       0         0  ...                      6.0   
        2016-05-19       0       0         0  ...                      7.0   
        2016-05-20       1       0         0  ...                      7.0   
        2016-05-21       0       0         1  ...                      7.0   
        2016-05-22       0       0         0  ...                      7.0   

                    demand__sum_values_week  demand__median_week  \
item_id date                                                       
0       2015-01-01                     46.0                  5.0   
        2015-01-02                     50.0                  5.0   
        2015-01-03                     49.0                  5.0   
        2015-01-04                     53.0                  5.0   
        2015-01-05                     52.0                  5.0   
...                                     ...                  ...   
49      2016-05-18                    115.0                 18.0   
        2016-05-19                    111.0                 16.0   
        2016-05-20                    118.0                 18.0   
        2016-05-21                    119.0                 18.0   
        2016-05-22                    112.0                 17.0   

                    demand__mean_week  demand__standard_deviation_week  \
item_id date                              

## Definition of the item_ids 


*   will be needed within the for loops later
*   will contain all 50 values (three values just for trying)



In [17]:
prods = [0,1,2]

## Determining some parameters
*   cu is calculated as the mean of the corresponding items sell_price
*   co is determined to be 60% less than cu
*   the potential params and their values for grid search are defined
*   the estimator, the scorer and finally the whole grid is defined





In [18]:
cu = round(X_train.loc[prods]['sell_price'].mean(),2)
co = round(X_train.loc[prods]['sell_price'].mean()*0.4,2)

k = np.array([7, 20, 30, 50, 60])
param_grid = dict(n_neighbors = k)

knn = KNeighborsWeightedNewsvendor(cu = cu, co = co)
scorer_pre = make_scorer(prescriptiveness_score, greater_is_better=True)
grid = GridSearchCV(knn, param_grid, cv=5, scoring = scorer_pre)

## Grid-search
*   an empty list is created
*   in a for-loop the grid search is executed for every item 
*   the best hyperparams are calculated and added to the corresponding list




In [19]:
best_param_k = []
for prod in prods:
  grid.fit(X_train.loc[prod], Y_train.loc[prod])
  k = next(iter(grid.best_params_.items()))[1]
  best_param_k.append(k)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)



In [20]:
result_knn = []
for prod in prods:
    knn = KNeighborsWeightedNewsvendor(cu = cu, co = co, n_neighbors=best_param_k[prod])
    knn.fit(X_train.loc[prod], Y_train.loc[prod])
    preds = knn.predict(X_test.loc[prod])
    avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co),2)
    result_knn.append(avc)

In [23]:
KNN_separate_avc = pd.DataFrame(result_knn, columns=['KNN_avc'])
KNN_separate_avc.index.name = 'item_id'

In [24]:
KNN_separate_avc

,KNN_avc
item_id,
0,2.79
1,1.88
2,1.98
